In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 熵权法计算权重

In [16]:
# 读取数据
data = pd.read_excel('../Resource/ExperimentDataBase/Spad_Dta_GRX.xlsx')

In [17]:
data_n_data = data[['SPAD','AGB','LAI']].values

In [18]:
data.head()

,SPAD,AGB,LAI,Pn,Ci,Tr,Gs
0,37.466667,1.708889,2.917709,16.573453,263.358997,3.381668,0.178455
1,35.991667,2.063889,2.054150,22.759493,268.242120,5.887681,0.395477
2,35.941667,1.997500,3.599002,16.718943,220.002423,4.542420,0.274608
3,34.941667,1.362222,3.320240,18.022922,196.736997,3.629510,0.170752
4,40.300000,1.965000,3.309667,17.883205,215.082727,4.303128,0.226866


In [19]:
def entropy_weight(matrix):
    """
    计算熵权法权重
    :param matrix: 决策矩阵，shape为(m, n)，m为样本数，n为指标数
    :return: 各指标的权重，shape为(n,)
    """
    m, n = matrix.shape
    # 使用归一化处理
    norm_matrix = (matrix - matrix.min(axis=0)) / (matrix.max(axis=0) - matrix.min(axis=0))
    #print(norm_matrix)
    # 计算信息熵
    norm_matrix[norm_matrix == 0] = 1e-10 # 将零值替换为一个较小的非零值
    entropy = - np.sum(norm_matrix * np.log(norm_matrix), axis=0)
    print(entropy)
    # 计算权重
    weight = (1 - entropy) / np.sum(1 - entropy)
    return weight

### 第一种归一化方式使用最大值和最小值进行缩放，通过除以最大值和最小值之间的差，将数据缩放到0到1之间，即：
- Xnorm = X-Xmin / Xmax - Xmin
- 其中x 是原始数据，Xnorm 是归一化后的数据，Xmin 和 Xmax  分别是该列的最小值和最大值。
### 第二种归一化方式是将每一列数据除以该列数据的和，从而将数据缩放到0到1之间，即：
- Xnorm  = X / Σ(i=1->n)Xi
- 其中 x 是原始数据， Xnorm 是归一化后的数据，n 是该列数据的数量。
> 第一种归一化方式将每个特征的范围缩放到相同的区间，但是可能会受到极值的影响。第二种归一化方式则保证了每个特征的和为1，不受极值的影响，但是可能会受到各特征数量的影响。

In [20]:
# 计算决策矩阵X和归一化矩阵Y
# X = data_n_data.iloc[:, :].values
# Y = np.apply_along_axis(lambda x: x / np.sum(x), 0, X)

In [21]:
# 计算信息熵和权重
# m, n = Y.shape
# E = (-1 / np.log(m)) * np.sum(Y * np.log(Y), axis=0)
# W = (1 - E) / np.sum(1 - E)

In [22]:
# 归一化处理
# W = W / np.sum(W)

In [23]:
# 输出结果
# for i, w in enumerate(W):
#     print(f"第{i+1}列的权重为{w:.3f}")

In [24]:
# 计算指标权重
weights = entropy_weight(data_n_data)
print(weights)

[20.80267728 16.78550941 19.11857991]
[0.36871848 0.29392031 0.33736121]


In [25]:
# 输出各指标权重
print("指标权重：")
for i in range(len(weights)):
    print("{}: {:.3f}".format(data.columns[i], weights[i]))

指标权重：
SPAD: 0.369
AGB: 0.294
LAI: 0.337


In [12]:
# 数据归一化 (极差变换法)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled =scaler.fit_transform(data_n_data)

In [13]:
import torch
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
var =torch.FloatTensor(scaled).to(device)

In [14]:
# 计算CGMIcv(综合长势检测指标)
# 定义计算公式函数
# 通过变异系数法计算的权重
weight_data = weights

#将numpy arry 转换为 Tensor(张量)
weight_data = torch.FloatTensor(weight_data).to(device)

cgmicv = var*weight_data

#最终结果
def finallyResult(data):
    resultArray = {}
    per_sum = 0
    for i in range(len(cgmicv)):
        for j in range(len(cgmicv[i])):
            per_sum = per_sum+cgmicv[i][j]
            # print("per_sum",per_sum)
        resultArray[i] = per_sum
        per_sum = 0
    return resultArray

dataS = finallyResult(cgmicv)
# print(dataS)

for i in range(len(dataS)):
    print(dataS[i])

tensor(0.4538)
tensor(0.3918)
tensor(0.5515)
tensor(0.3967)
tensor(0.5894)
tensor(0.5029)
tensor(0.4330)
tensor(0.8743)
tensor(0.5306)
tensor(0.6449)
tensor(0.6123)
tensor(0.5273)
tensor(0.5847)
tensor(0.6492)
tensor(0.7796)
tensor(0.3930)
tensor(0.2138)
tensor(0.4394)
tensor(0.4436)
tensor(0.5477)
tensor(0.6325)
tensor(0.4362)
tensor(0.6238)
tensor(0.5464)
tensor(0.5278)
tensor(0.7501)
tensor(0.7712)
tensor(0.5358)
tensor(0.5120)
tensor(0.4967)
tensor(0.6090)
tensor(0.4661)
tensor(0.5084)
tensor(0.6536)
tensor(0.5276)
tensor(0.4204)
tensor(0.7140)
tensor(0.6387)
tensor(0.4124)
tensor(0.4579)
tensor(0.4565)
tensor(0.3979)
tensor(0.1076)
tensor(0.0240)
tensor(0.2599)
tensor(0.3414)
tensor(0.4392)
tensor(0.4184)
tensor(0.4445)
tensor(0.5751)
tensor(0.3153)
tensor(0.6061)
tensor(0.4725)
tensor(0.4055)
tensor(0.2754)
tensor(0.3438)
tensor(0.2547)
tensor(0.2885)
tensor(0.4378)
tensor(0.3635)
tensor(0.4941)
tensor(0.4441)
tensor(0.5427)
tensor(0.3882)
tensor(0.3723)
tensor(0.4976)
